In [49]:
import sys
sys.path.append('../')

import json
from glob import glob
import os
from tqdm import tqdm
import numpy as np

from EM import NiftiManager, Evaluate, EM, ElastixTransformix, FileManager, BrainAtlasManager
from utils.fusion import majority_voting_fusion, weighted_voting_fusion, staple_fusion
from utils.evaluate import hausdorff_distance

# To allow auto reload to this notebook after modifying any external file imported
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
NM    = NiftiManager()
EVAL  = Evaluate()
ET    = ElastixTransformix()
FM    = FileManager()
BM    = BrainAtlasManager()

In [36]:
print(os.listdir('./output_base/'))
print(os.listdir('./output_base/images'))
print(os.listdir('./output_base/labels'))

['images', 'labels']
['output_IBSR_11', 'output_IBSR_12', 'output_IBSR_13', 'output_IBSR_14', 'output_IBSR_17']
['output_IBSR_11', 'output_IBSR_12', 'output_IBSR_13', 'output_IBSR_14', 'output_IBSR_17']


In the next few cells, we will run the fusion and evaluation using registered intensities and labelled that were registered to all val/test subjects, not taking into account the group or any other information that were analyzed in the EDA notebook.

In [55]:
# {fixed}/{moving_folders}
# each fixed will be the test subject (for now, it is the validation), thus we will validate separately

output_images_dir = './output_base/images'
output_labels_dir = './output_base/labels'

# iterate through all fusion techniques
for fusion_callback in [majority_voting_fusion, weighted_voting_fusion, staple_fusion]:
    print(f'Evaluating using "{fusion_callback.__name__}" fusion technique')

    # iterate through all fixed directories
    for fixed_intensity, fixed_label in zip(os.listdir(output_images_dir), os.listdir(output_labels_dir)):
        # create a path for the current fixed
        fixed_label_path = os.path.join(output_labels_dir, fixed_label)
        fixed_intensity_path = os.path.join(output_labels_dir, fixed_intensity)
        
        # get the propagated masks that were result of the label propagation to the current fixed
        moving_masks = [m.replace('\\', '/') for m in glob(os.path.join(fixed_label_path, "IBSR_**_seg", "*.nii"), recursive=True)]
        moving_intensity = [m.replace('\\', '/') for m in glob(os.path.join(fixed_intensity_path, "IBSR_**", "*.nii"), recursive=True)]
        
        # load the label gt of the test image (target) ( validation for now )
        fixed_filename = fixed_intensity.split('output_')[-1] # output_IBSR_11 > IBSR_11
        target_label_path = f'../TrainingValidationTestSets/Validation_Set/{fixed_filename}/{fixed_filename}_seg.nii.gz'
        target_intensity_path = f'../TrainingValidationTestSets/Validation_Set/{fixed_filename}/{fixed_filename}.nii.gz'
    
        target_label_nifti = NM.load_nifti(target_label_path)[0][:,:,:,0]
        target_intensity_nifti = NM.load_nifti(target_intensity_path)[0][:,:,:,0]
    
        # fuse method
        if fusion_callback.__name__ in ['majority_voting_fusion', 'staple_fusion']:
            mean_csf, mean_gm, mean_wm = fusion_callback(
                labels_dirs = moving_masks, 
                load_nifti_callback = NM.load_nifti)
            
        elif fusion_callback.__name__ == 'weighted_voting_fusion':
            mean_csf, mean_gm, mean_wm = fusion_callback(
                labels_dirs = moving_masks, 
                intensity_dirs = moving_intensity,
                target_intensity_path = target_intensity_path,
                load_nifti_callback = NM.load_nifti)
        
        # segment using the tissue atlases we created using the voting technique
        segmentation_result, concatenated_atlas = BM.segment_using_tissue_atlas(target_intensity_nifti, target_label_nifti, mean_csf, mean_gm, mean_wm)
    
        # evaluate dice score
        dice_coefficients = EVAL.evaluate_dice_volumes(segmentation_result, target_label_nifti, labels={'BG':0, 'CSF':1, 'GM':2, 'WM':3})
        # hausdorff_distances = hausdorff_distance(segmentation_result, target_label_nifti, labels={'BG':0, 'CSF':1, 'GM':2, 'WM':3}, voxelspacing=...)
        
        print(f'Testing volume {fixed_filename}:-')
        print(f' - Dice scores {dice_coefficients}')
        # print(f' - Hausdorff Distance: {hausdorff_distances}')
    
    print('------------------------------------------------------------------------------------------------------------------------------------------------------')

Evaluating using "majority_voting_fusion" fusion technique
Testing volume IBSR_11:-
 - Dice scores {'WM': 0.78578, 'GM': 0.876239, 'CSF': 0.740233}
Testing volume IBSR_12:-
 - Dice scores {'WM': 0.800773, 'GM': 0.900176, 'CSF': 0.746292}
Testing volume IBSR_13:-
 - Dice scores {'WM': 0.787228, 'GM': 0.90687, 'CSF': 0.585436}
Testing volume IBSR_14:-
 - Dice scores {'WM': 0.805461, 'GM': 0.902183, 'CSF': 0.793226}
Testing volume IBSR_17:-
 - Dice scores {'WM': 0.780982, 'GM': 0.900373, 'CSF': 0.83405}
------------------------------------------------------------------------------------------------------------------------------------------------------
Evaluating using "weighted_voting_fusion" fusion technique
Testing volume IBSR_11:-
 - Dice scores {'WM': 0.792716, 'GM': 0.875556, 'CSF': 0.747126}
Testing volume IBSR_12:-
 - Dice scores {'WM': 0.800848, 'GM': 0.895985, 'CSF': 0.724778}
Testing volume IBSR_13:-
 - Dice scores {'WM': 0.783434, 'GM': 0.901706, 'CSF': 0.582829}
Testing volume

As we observed different voxel sizes accross the dataset, we defined a dataset structure for handling the registration paths, where we will register similar modalities with similar voxel sizes togather to avoid sampling. In `TrainingValidationTestSets\description.json`, we clustered each given volume to a group and each group will be registered togather.

In [54]:
# {fixed}/{moving_folders}
# each fixed will be the test subject (for now, it is the validation), thus we will validate separately

output_images_dir = './output/images'
output_labels_dir = './output/labels'

# iterate through all fusion techniques
for fusion_callback in [majority_voting_fusion, weighted_voting_fusion, staple_fusion]:
    print(f'Evaluating using "{fusion_callback.__name__}" fusion technique')

    # iterate through all fixed directories
    for fixed_intensity, fixed_label in zip(os.listdir(output_images_dir), os.listdir(output_labels_dir)):
        # create a path for the current fixed
        fixed_label_path = os.path.join(output_labels_dir, fixed_label)
        fixed_intensity_path = os.path.join(output_labels_dir, fixed_intensity)
        
        # get the propagated masks that were result of the label propagation to the current fixed
        moving_masks = [m.replace('\\', '/') for m in glob(os.path.join(fixed_label_path, "IBSR_**_seg", "*.nii"), recursive=True)]
        moving_intensity = [m.replace('\\', '/') for m in glob(os.path.join(fixed_intensity_path, "IBSR_**", "*.nii"), recursive=True)]
        
        # load the label gt of the test image (target) ( validation for now )
        fixed_filename = fixed_intensity.split('output_')[-1] # output_IBSR_11 > IBSR_11
        target_label_path = f'../TrainingValidationTestSets/Validation_Set/{fixed_filename}/{fixed_filename}_seg.nii.gz'
        target_intensity_path = f'../TrainingValidationTestSets/Validation_Set/{fixed_filename}/{fixed_filename}.nii.gz'
    
        target_label_nifti = NM.load_nifti(target_label_path)[0][:,:,:,0]
        target_intensity_nifti = NM.load_nifti(target_intensity_path)[0][:,:,:,0]
    
        # fuse method
        if fusion_callback.__name__ in ['majority_voting_fusion', 'staple_fusion']:
            mean_csf, mean_gm, mean_wm = fusion_callback(
                labels_dirs = moving_masks, 
                load_nifti_callback = NM.load_nifti)
            
        elif fusion_callback.__name__ == 'weighted_voting_fusion':
            mean_csf, mean_gm, mean_wm = fusion_callback(
                labels_dirs = moving_masks, 
                intensity_dirs = moving_intensity,
                target_intensity_path = target_intensity_path,
                load_nifti_callback = NM.load_nifti)
        
        # segment using the tissue atlases we created using the voting technique
        segmentation_result, concatenated_atlas = BM.segment_using_tissue_atlas(target_intensity_nifti, target_label_nifti, mean_csf, mean_gm, mean_wm)
    
        # evaluate dice score
        dice_coefficients = EVAL.evaluate_dice_volumes(segmentation_result, target_label_nifti, labels={'BG':0, 'CSF':1, 'GM':2, 'WM':3})
        # hausdorff_distances = hausdorff_distance(segmentation_result, target_label_nifti, labels={'BG':0, 'CSF':1, 'GM':2, 'WM':3})
        
        print(f'Testing volume {fixed_filename}:-')
        print(f' - Dice scores {dice_coefficients}')
        # print(f' - Hausdorff Distance: {hausdorff_distances}')
    
    print('------------------------------------------------------------------------------------------------------------------------------------------------------')

Evaluating using "majority_voting_fusion" fusion technique
Testing volume IBSR_11:-
 - Dice scores {'WM': 0.783063, 'GM': 0.849518, 'CSF': 0.556359}
Testing volume IBSR_12:-
 - Dice scores {'WM': 0.775957, 'GM': 0.864478, 'CSF': 0.632066}
Testing volume IBSR_13:-
 - Dice scores {'WM': 0.774438, 'GM': 0.894708, 'CSF': 0.533588}
Testing volume IBSR_14:-
 - Dice scores {'WM': 0.798197, 'GM': 0.892459, 'CSF': 0.735868}
Testing volume IBSR_17:-
 - Dice scores {'WM': 0.72679, 'GM': 0.885204, 'CSF': 0.618607}
------------------------------------------------------------------------------------------------------------------------------------------------------
Evaluating using "weighted_voting_fusion" fusion technique
Testing volume IBSR_11:-
 - Dice scores {'WM': 0.781398, 'GM': 0.847654, 'CSF': 0.561203}
Testing volume IBSR_12:-
 - Dice scores {'WM': 0.773564, 'GM': 0.862476, 'CSF': 0.63342}
Testing volume IBSR_13:-
 - Dice scores {'WM': 0.773911, 'GM': 0.894359, 'CSF': 0.529962}
Testing volum

code below is just for debugging, don't delete

In [48]:
# labels_path_output_IBSR_11 = './output/labels/output_IBSR_11/'
# images_path_output_IBSR_11 = './output/images/output_IBSR_11/'

# labels_output_IBSR_11 = glob(os.path.join(labels_path_output_IBSR_11, "IBSR_**_seg", "*.nii"), recursive=True)
# images_output_IBSR_11 = glob(os.path.join(images_path_output_IBSR_11, "IBSR_**", "*.nii"), recursive=True)

In [49]:
# labels_output_IBSR_11

In [50]:
# nn = NM.load_nifti(labels_output_IBSR_11[0])[0]
# NM.show_nifti(nn, "test", 140)

In [51]:
# mean_csf, mean_gm, mean_wm = staple_fusion(
#     labels_dirs = labels_output_IBSR_11,
#     load_nifti_callback = NM.load_nifti
# )

In [52]:
# NM.show_nifti(mean_wm, "staple_res", 140)

In [11]:
# # segmenting using those atlases
# IBSR_11 = '../TrainingValidationTestSets/Validation_Set/IBSR_11/IBSR_11.nii.gz'
# IBSR_11_label = '../TrainingValidationTestSets/Validation_Set/IBSR_11/IBSR_11_seg.nii.gz'

In [12]:
# mean_csf, mean_gm, mean_wm = weighted_voting(
#     labels_dirs = labels_output_IBSR_11, 
#     intensity_dirs = images_output_IBSR_11,
#     target_intensity_path = IBSR_11,
#     load_nifti_callback = NM.load_nifti)

# print(mean_csf.shape, mean_gm.shape, mean_wm.shape)


In [13]:
# NM.show_mean_volumes(mean_csf, mean_wm, mean_gm, slices=[128])

In [5]:
# with open('../TrainingValidationTestSets/description.json', 'r') as json_file:
#         dictionary = json.loads(json_file.read())
# # FM.pprint_objects(dictionary)

# for modality in dictionary:
#     # we get the data of the same modality
#     dataset = dictionary[modality]
    
#     # register the data of the same modality to the target
#     # make sure the code is reusable for inference, now we will evaluate the fusions on the validation set, but for submission, 
#     # it has to be submitted on the test set
#     print(dataset['train'], dataset['valid'])

In [56]:
# labels_path_output_IBSR_11 = './output/labels/output_IBSR_11/'
# images_path_output_IBSR_11 = './output/images/output_IBSR_11/'

# labels_output_IBSR_11 = glob(os.path.join(labels_path_output_IBSR_11, "IBSR_**_seg", "*.nii"), recursive=True)
# images_output_IBSR_11 = glob(os.path.join(images_path_output_IBSR_11, "IBSR_**", "*.nii"), recursive=True)

In [57]:
# labels_output_IBSR_11

In [31]:
# import nibabel as nib
# import numpy as np
# from skimage.util import view_as_windows
# from scipy.ndimage import label

# def extract_patches(image, patch_size):
#     return view_as_windows(image, patch_size)

# def compute_similarity(patch1, patch2):
#     # Implement your similarity metric here
#     # Example: Normalized Cross-Correlation
#     return np.sum(patch1 * patch2) / (np.sqrt(np.sum(patch1 ** 2)) * np.sqrt(np.sum(patch2 ** 2)))

# def non_local_fusion(image, labels, patch_size):
#     # Extract patches from the image and labels
#     image_patches = extract_patches(image, patch_size)
#     label_patches = extract_patches(labels, patch_size)

#     fused_labels = np.zeros_like(labels)

#     for i in tqdm(range(image.shape[0])):
#         for j in range(image.shape[1]):
#             for k in range(image.shape[2]):
#                 center_patch = image_patches[i, j, k]
#                 center_label = label_patches[i, j, k]

#                 # Accumulate the weighted sum
#                 weighted_sum = 0.0
#                 total_similarity = 0.0

#                 for m in range(image_patches.shape[0]):
#                     for n in range(image_patches.shape[1]):
#                         for o in range(image_patches.shape[2]):
#                             # Adjust indices based on the shape of image_patches
#                             neighbor_patch = image_patches[m, n, o, :, :, :]
#                             similarity = compute_similarity(center_patch, neighbor_patch)
#                             weighted_sum += similarity * label_patches[m, n, o, :, :, :]
#                             total_similarity += similarity

#                 # Normalize and assign the mean result
#                 if total_similarity > 0:
#                     fused_labels[i, j, k] = np.mean(accumulated_labels, axis=0) / total_similarity

#     return fused_labels



In [33]:
# seg1 = NM.load_nifti(labels_output_IBSR_11[0])[0]
# seg2 = NM.load_nifti(labels_output_IBSR_11[1])[0]

# patch_size = (20, 20, 20)  # Adjust patch size based on your requirements

# fused_labels = non_local_fusion(seg1, seg2, patch_size)

# # # Extract patches from the image and labels
# # image_patches = extract_patches(seg1, patch_size)
# # # label_patches = extract_patches(labels, patch_size)

# # print(image_patches.shape)